In [1]:
import os
import shutil
import argparse
import logging # TBK: logging module
import logging.config # TBK
import configparser # TBK: To read config file
import tqdm # TBK
from time import time
import psutil
from multiprocessing import Pool
import datetime
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os
from PIL import Image
import pandas as pd

In [2]:
def classify(model_file, input_dir):
    model = tf.keras.models.load_model(model_file)

    pad(input_dir)
    images = []
    image_files = []
    for img in os.listdir(input_dir):
        image_files.append(img)
        img = tf.keras.preprocessing.image.load_img(input_dir+img, target_size=(128, 128), color_mode='grayscale')
        #img = img.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        images.append(img)
    images = np.vstack(images)


    predictions = model.predict(images)
    prediction_labels = np.argmax(predictions, axis=-1)
    np.savetxt('prediction.csv', predictions, delimiter=',')

    with open('prediction_names.csv', newline='', mode='w') as csvfile:
        csvwritter = csv.writer(csvfile, delimiter='\n')
        csvwritter.writerow(image_files)

In [3]:
def init_model(num_classes, img_height, img_width):

    #strategy = tf.distribute.MirroredStrategy()
    #with strategy.scope():
    model = ResNet18([img_height, img_width, 1], config['training']['model_name'], num_classes)
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
    model.summary()
    return(model)

In [4]:
def ResNet18(input_shape, name, num_classes):
    BN_AXIS = 3

    img_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Rescaling(-1. / 255, 1)(img_input)
    x = tf.keras.layers.RandomRotation(2)(x)
    x = tf.keras.layers.RandomFlip("horizontal_and_vertical")(x)

    x = tf.keras.layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(x)
    x = tf.keras.layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = tf.keras.layers.BatchNormalization(axis=BN_AXIS, name='bn_conv1')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = make_basic_block_layer(x, filter_num=64, blocks=2)
    x = make_basic_block_layer(x, filter_num=128, blocks=2, stride=2)
    x = make_basic_block_layer(x, filter_num=256, blocks=2, stride=2)
    x = make_basic_block_layer(x, filter_num=512, blocks=2, stride=2)


    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(img_input, x, name=name)
    return model


def make_basic_block_base(inputs, filter_num, stride=1):
    BN_AXIS = 3
    x = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=(3, 3),
                                        strides=stride,
                                        kernel_initializer='he_normal',
                                        padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization(axis=BN_AXIS)(x)
    x = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=(3, 3),
                                        strides=1,
                                        kernel_initializer='he_normal',
                                        padding="same")(x)
    x = tf.keras.layers.BatchNormalization(axis=BN_AXIS)(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    shortcut = inputs
    if stride != 1:
        shortcut = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(1, 1),
                                            strides=stride,
                                            kernel_initializer='he_normal')(inputs)
        shortcut = tf.keras.layers.BatchNormalization(axis=BN_AXIS)(shortcut)

    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)

    return x

def make_basic_block_layer(inputs, filter_num, blocks, stride=1):
    x = make_basic_block_base(inputs, filter_num, stride=stride)

    for _ in range(1, blocks):
        x = make_basic_block_base(x, filter_num, stride=1)

    return x

In [5]:
def load_model(config):
    #if int(config['training']['start']) > 0:
    #    return(tf.keras.models.load_model(config['training']['training_dir'], config))
    
    return(init_model(109, int(config['training']['image_size']), int(config['training']['image_size'])))
    

In [6]:
def train_model(model, config, train_ds, val_ds):
    history = model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=int(config['training']['stop'])-int(config['training']['start']),
                        initial_epoch=int(config['training']['start']),
                        batch_size = int(config['training']['batchsize']))
    
    
    return(model)

In [7]:
def init_ts(config):
    train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
        config['training']['scnn_dir'] + '/data',
        interpolation='area',
        validation_split = 0.2,
        subset = "both",
        seed = 123,
        image_size = (int(config['training']['image_size']), int(config['training']['image_size'])),
        batch_size = int(config['training']['batchsize']),
        color_mode = 'grayscale')
    return(train_ds, val_ds)

In [15]:
config = {
    'training' : {
        'scnn_dir': '../../training/20231002',
        'model_name': 'Gamma',
        'model_path': '../../model/',
        'image_size': '128',
        'start': 0,
        'stop': 10,
        'validationSetRatio': 0.2,
        'batchsize': 16,
        'seed': 123
    }
}


In [13]:
v_string = "V2023.10.09"
print(f"Starting CNN Model Training Script {v_string}")
    
train_ds, val_ds = init_ts(config)

Starting CNN Model Training Script V2023.10.09
Found 19336 files belonging to 109 classes.
Using 15469 files for training.
Using 3867 files for validation.
Model: "Gamma"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 128, 128, 1)  0           ['input_1[0][0]']                
                                                                                                  
 random_rotation (RandomRotatio  (None, 128, 128, 1)  0          ['rescaling[0][0]']              
 n)                                  

                                                                                                  
 add_2 (Add)                    (None, 16, 16, 128)  0           ['dropout_2[0][0]',              
                                                                  'batch_normalization_6[0][0]']  
                                                                                                  
 activation_3 (Activation)      (None, 16, 16, 128)  0           ['add_2[0][0]']                  
                                                                                                  
 conv2d_7 (Conv2D)              (None, 16, 16, 128)  147584      ['activation_3[0][0]']           
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 16, 16, 128)  512        ['conv2d_7[0][0]']               
 rmalization)                                                                                     
          

                                                                                                  
 conv2d_17 (Conv2D)             (None, 4, 4, 512)    2359808     ['activation_7[0][0]']           
                                                                                                  
 batch_normalization_17 (BatchN  (None, 4, 4, 512)   2048        ['conv2d_17[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_18 (Conv2D)             (None, 4, 4, 512)    2359808     ['batch_normalization_17[0][0]'] 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 4, 4, 512)   2048        ['conv2d_18[0][0]']              
 ormalization)                                                                                    
          

In [ ]:
model = load_model(config)

In [ ]:
model = train_model(model, config, train_ds, val_ds)

Epoch 1/10
967/967 [==============================] - 59s 50ms/step - loss: 2.9911 - accuracy: 0.2097 - val_loss: 4.2478 - val_accuracy: 0.1231
Epoch 2/10
967/967 [==============================] - 42s 44ms/step - loss: 2.1866 - accuracy: 0.3596 - val_loss: 4.7554 - val_accuracy: 0.1660
Epoch 3/10
967/967 [==============================] - 43s 45ms/step - loss: 1.8908 - accuracy: 0.4238 - val_loss: 1.7310 - val_accuracy: 0.4784
Epoch 4/10
967/967 [==============================] - 46s 47ms/step - loss: 1.6942 - accuracy: 0.4781 - val_loss: 3.8426 - val_accuracy: 0.2708
Epoch 5/10
525/967 [===============>..............] - ETA: 19s - loss: 1.5640 - accuracy: 0.5119

In [12]:
model.save(config['training']['model_path'] + '/' + config['training']['model_name'])
    
predictions = model.predict(val_ds)
predictions = np.argmax(predictions, axis = -1)
y = np.concatenate([y for x, y in val_ds], axis=0)

confusion_matrix = tf.math.confusion_matrix(y, predictions)
confusion_matrix = pd.DataFrame(confusion_matrix, index = train_ds.class_names, columns = train_ds.class_names)
confusion_matrix.to_csv(config['training']['model_path'] + '/' + config['training']['model_name'] + ' confusion.csv')
    
summary = {
    "file": val_ds.file_paths,
    "label": val_ds.class_names,
    "prediction": pd.Series(predictions),
}
    
summary = pd.DataFrame(summary)
summary.to_csv(config['training']['model_path'] + '/' + config['training']['model_name'] + ' summary.csv')

INFO:tensorflow:Assets written to: ../../model//Beta/assets


INFO:tensorflow:Assets written to: ../../model//Beta/assets


484/484 [==============================] - 8s 16ms/step


ValueError: All arrays must be of the same length

In [ ]:
config['training']['model_name']